## Aprendendo a teoria do Sha512

In [1]:
from hash.attr_helper import K_512, init_state_512
from hash.utils_helper import RotR, ShR

In [2]:
blocksize = 128
pad = 0xffffffffffffffff

In [3]:
Ch = lambda x, y, z: (x & y) ^ (~x & z)
Maj = lambda x, y, z: (x & y) ^ (x & z) ^ (y & z)
sigma0 = lambda x: RotR(x, 28) ^ RotR(x, 34) ^ RotR(x, 39)
sigma1 = lambda x: RotR(x, 14) ^ RotR(x, 18) ^ RotR(x, 41)
gamma0 = lambda x: RotR(x, 1) ^ RotR(x, 8) ^ ShR(x, 7)
gamma1 = lambda x: RotR(x, 19) ^ RotR(x, 61) ^ ShR(x, 6)

In [4]:
ctx = {
    'data': None,
    'datalen': 0,
    'bitsize': 0,
    'state': None
}

def sha_init(ctx):
    ctx['data'] = [0] * blocksize
    ctx['datalen'] = 0
    ctx['bitsize'] = 0
    ctx['state'] = list(init_state_512)

In [5]:
def sha_transform(ctx):
    w = [0] * 80
    m = ctx['data']
    for i in range(16):
        j = i * 8
        w[i] = (m[j] << 56 | m[j+1] << 48 | m[j+2] << 40 | 
                m[j+3] << 32 | m[j+4] << 24 | m[j+5] << 16 | 
                m[j+6] << 8 | m[j+7]) & pad
    
    
    for i in range(16, 80):
        w[i] = gamma1(w[i-2]) + w[i-7] + gamma0(w[i-15]) + w[i-16]

        a,b,c,d,e,f,g,h = ctx['state']
    
    for i in range(80):
        T1 = h + sigma1(e) + Ch(e,f,g) + K_512[i] + w[i]
        T2 = sigma0(a) + Maj(a,b,c)
        h = g
        g = f
        f = e
        e = d + T1
        d = c
        c = b
        b = a
        a = T1 + T2

    state = (a,b,c,d,e,f,g,h)
    for index, s in enumerate(state):
        ctx['state'][index] = (ctx['state'][index] + s) & pad

In [15]:
def sha_update(ctx, msg):
    for letter in msg:
        ctx['data'][ctx['datalen']] = ord(letter)
        ctx['datalen'] += 1
        if ctx['datalen'] == blocksize:
            sha_transform(ctx)
            ctx['data'] = [0] * blocksize
            ctx['bitsize'] += blocksize * 8
            ctx['datalen'] = 0


In [13]:
def sha_final(ctx):
    i = ctx['datalen']
    ctx['data'][i] = 0x80
    i += 1

    if i > 112:
        sha_transform(ctx)
        ctx['data'] = [0] * blocksize

    ctx['bitsize'] += ctx['datalen'] * 8
    ctx['data'][112] = (ctx['bitsize'] >> 120) & 0xff
    ctx['data'][113] = (ctx['bitsize'] >> 112) & 0xff
    ctx['data'][114] = (ctx['bitsize'] >> 104) & 0xff
    ctx['data'][115] = (ctx['bitsize'] >> 96) & 0xff
    ctx['data'][116] = (ctx['bitsize'] >> 88) & 0xff
    ctx['data'][117] = (ctx['bitsize'] >> 80) & 0xff
    ctx['data'][118] = (ctx['bitsize'] >> 72) & 0xff
    ctx['data'][119] = (ctx['bitsize'] >> 64) & 0xff
    ctx['data'][120] = (ctx['bitsize'] >> 56) & 0xff
    ctx['data'][121] = (ctx['bitsize'] >> 48) & 0xff
    ctx['data'][122] = (ctx['bitsize'] >> 40) & 0xff
    ctx['data'][123] = (ctx['bitsize'] >> 32) & 0xff
    ctx['data'][124] = (ctx['bitsize'] >> 24) & 0xff
    ctx['data'][125] = (ctx['bitsize'] >> 16) & 0xff
    ctx['data'][126] = (ctx['bitsize'] >> 8) & 0xff
    ctx['data'][127] = ctx['bitsize']
    sha_transform(ctx)

In [8]:
class Sha512:
    def __init__(self, message=''):
        sha_init(ctx)
        self.update(message)
    def update(self, message):
        sha_update(ctx, message)
        sha_final(ctx)
        
    def hexdigest(self):
        return ''.join([hex(s)[2:] for s in ctx['state']])

In [16]:
t = 'this is a test!'
print(Sha512().hexdigest() == 'cf83e1357eefb8bdf1542850d66d8007d620e4050b5715dc83f4a921d36ce9ce47d0d13c5d85f2b0ff8318d2877eec2f63b931bd47417a81a538327af927da3e')
print(Sha512(t).hexdigest() == '5746df6112981b3236c15055113e14809578b42d9307f3dd21ef628cb2c78d6ca3f191c2402d94f98892729c41b4e3f97620a893cd0481a2846c9c166ed0e6de')
print(Sha512(t*10).hexdigest() == '83d57e91db26823110d7c3b387f457b71548458809227d60cf298ab19da67c6cb91f9ed34ead4b659174ac3f753ae8e99a5ebbfa0c48659fdf2445e16dadf17d')

s = Sha512(t)
print(s.hexdigest() == '5746df6112981b3236c15055113e14809578b42d9307f3dd21ef628cb2c78d6ca3f191c2402d94f98892729c41b4e3f97620a893cd0481a2846c9c166ed0e6de')
print(s.hexdigest() == '5746df6112981b3236c15055113e14809578b42d9307f3dd21ef628cb2c78d6ca3f191c2402d94f98892729c41b4e3f97620a893cd0481a2846c9c166ed0e6de')
s.update(t)
print(s.hexdigest() == '7693c3358b77f120c99b964578d4f338bc29f209cc78a3ac43727e72eb7f1472ddeddfafd8288fab50d82af5d44bfbb38ccbc960d37a7fef0a719e7bda14a1ee')


True
True
True
True
True
False
